In [3]:
import numpy as np
from numpy.random import rand
import math
from random import randint
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
class ConnectN:
    """
    Connect N game simulator for two players, 1 and -1.
    
    Inputs:
    Grid size- creates a grid size x grid size square board
    N- number of tokens a player must connect to win the game
    
    """
    
    def __init__(self, grid_size, n):
        self.n = n
        self.grid_size = grid_size
        self.grid = np.zeros([grid_size,grid_size])
        self.finished = 0
        self.turn_num = 0
        
    def reset(self):
        self.__init__(self.grid_size, self.n)

    def check_win(self, col, row, player):
        """
        Checks if given player has connected N tokens.
        """
        for i in range(0, self.n):
            if sum(self.grid[col, row - i:row - i + self.n]) == self.n*player:
                self.finished = 1
                return 1
            if sum(self.grid[col - i: col - i + self.n, row]) == self.n*player:
                self.finished = 1
                return 1
            if col - i >= 0 and col - i + self.n - 1 < self.grid_size and row - i >= 0 and row - i + self.n - 1 < self.grid_size:
                if sum([self.grid[col - i + x, row - i + x] for x in range(0, self.n)]) == self.n*player:
                    self.finished = 1
                    return 1
            if col - i >= 0 and col - i + self.n - 1 < self.grid_size and row + i >= self.n - 1 and row + i < self.grid_size:
                if sum([self.grid[col - i + x, row + i - x] for x in range(0, self.n)]) == self.n*player:
                    self.finished = 1
                    return 1
        return 0

    def move(self, col, player):
        """
        Given player and column to move in, modifies board and increments the turn counter.
        
        Returns a tuple, where first value is return message and second value is reward.
        """
        self.turn_num += 1
        
        if self.finished == 1:
            return 1, 50
        sum_col = np.sum([abs(x) for x in self.grid[col]])
        if sum_col == self.grid_size:
            return -1, -1
        self.grid[col, sum_col] = player
        if self.check_win(col, sum_col, player) == 1:
            return 1, 50
        return 0, 0
    
    def turn(self):
        """
        Returns which player's turn it is. First turn is player 1, second turn is player -1.
        """
        if self.turn_num%2 == 0:
            return 1
        else:
            return -1
        
    def next_possible_moves(self):
        """
        Returns array of possible columns for a next move
        """
        columns = []
        
        for i in xrange(0, self.grid_size):
            if (0 in self.grid[i]):
                columns.append(i)
                
        return columns
    
    def all_tokens_placed(self):
        """
        Returns location of all tokens (column, row) that have been placed
        """
        all_tokens = []
        
        for col in xrange(0, self.grid_size):
            for row in xrange(0, self.grid_size): 
                if self.grid[col][row] != 0:
                    all_tokens.append({"location": [col, row], "player": self.grid[col][row]})
                    
        return all_tokens
    
    def is_empty(self, col, row):
        """
        Returns if a given spot (column, row) is empty
        """
        return self.grid[col][row] == 0
    
    def print_grid(self):
        print(np.rot90(self.grid))

In [5]:
def grid_to_key(grid):
    """
    Converts ConnectN grid into string for dict indexing
    """

    key = ""

    for row in np.rot90(grid):
        for column in row:
            key += str(int(column))

    return key

In [6]:
class ConnectDict(dict):
    """
    Creates a Custom Dict that inherits from Python's native dict.
    Takes in a number of states.
    Adds keys to dict each time lookup is necessary to avoid full dict initialization.
    """
    
    def __init__(self, num_states, *arg, **kw):
        self.num_states = num_states
        super(ConnectDict, self).__init__(*arg, **kw)
        
    def __getitem__(self, key):
        
        if key not in self.__dict__:
            dict.__setitem__(self, key, np.zeros(self.num_states))
        return dict.__getitem__(self, key)

In [7]:
class TD_Learner(object):
    """
    Base class for Temporal Difference Learners, like Sarsa and Q learning.
    """
    
    def __init__(self, task, epsilon=.1, discount_factor=.9, learning_rate=.5):
        
        self.num_states = task.grid_size
        self.num_actions = task.grid_size
        self.epsilon = epsilon
        self.discount_factor = discount_factor
        self.learning_rate = learning_rate
        self.value_table = ConnectDict(self.num_states)
        self.reset()

    def reset(self):
        self.last_board_state = None
        self.last_action = None
        self.value_table = ConnectDict(self.num_states)
    
    def softmax(self, next_board_state):
        """
        Implementation of Softmax Policy, which weights towards better actions rather
        than sampling uniformly across all possible actions (epsilon-greedy)
        """
        
        def weighted_pick(weights,n_picks):
            t = np.cumsum(weights)
            s = sum(weights)
            return np.searchsorted(t,rand(n_picks)*s)

        tau = .5
        num = math.e**(self.value_table[grid_to_key(next_board_state)]/tau)
        probs = num/sum(num)
        best_action = weighted_pick(probs, 1)

        return best_action[0]
        

In [20]:
class Q_Learner(TD_Learner):
    """
    Implementation of Q Learning, inheriting from TD Learner base class. 
    """
    
    def __init__(self, task, epsilon=.1, discount_factor=.9, learning_rate=.5):   
        TD_Learner.__init__(self, task, epsilon, discount_factor, learning_rate) 
                                                

    def interact(self, reward, next_board_state):

        if reward is None:
            next_action = self.softmax(next_board_state)

            self.last_board_state = next_board_state
            self.last_action = next_action
            return self.last_action
                
        if reward == 50:
            delta = delta = reward - self.value_table[grid_to_key(self.last_board_state)][self.last_action]
            self.value_table[grid_to_key(self.last_board_state)][self.last_action] += self.learning_rate * delta
            return self.last_action
        
        """
        VDBE-Softmax policy. If draw < epsilon, perform Softmax. Else do best action.
        """
        draw = np.random.uniform(0,1,1)

        if draw < self.epsilon:
            next_action = self.softmax(next_board_state)
        else:
            next_action = np.argmax(self.value_table[grid_to_key(next_board_state)])

        # Update value function.
        delta = reward + self.discount_factor * np.amax(self.value_table[grid_to_key(next_board_state)]) - self.value_table[grid_to_key(self.last_board_state)][self.last_action]
        self.value_table[grid_to_key(self.last_board_state)][self.last_action] += self.learning_rate * delta

        # Eligibility traces
#         for i in range(0, self.num_states):
#             for j in range(0, self.num_actions):
#                 self.value_table[i, j] += this_iter_learning_rate * delta * self.e[i,j]
#                 if self.last_action == j:
#                     self.e[i, j] = self.discount_factor * self.trace_size * self.e[i,j]
#                 else:
#                     self.e[i, j] = 0
                    
        self.last_board_state = next_board_state
        self.last_action = next_action

        return self.last_action


In [21]:
task = ConnectN(7, 4)
agent = Q_Learner(x)

In [22]:
run_trial(agent, 10, 1, 1)

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]]
Won!
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]

([0, 50, 0, 0, 50, 0, 0, 50, 0, 0],
 [50],
 [0, 50, 50, 50, 100, 100, 100, 150, 150, 150],
 [50])

In [18]:
def run_trial(agent, MIN_ITERATIONS, MIN_EPISODES, player):
    """
    Runs the ConnectN Simulator for one player given an agent and number of iterations and episodes
    """
    
    rewards_by_iteration = []
    rewards_by_episode = []
    cumu_rewards_by_iteration = []
    cumu_rewards_by_episode = []
    
    iteration = episode = 0
    agent.reset()

    while iteration < MIN_ITERATIONS or episode < MIN_EPISODES:
        
        task.reset()
        board_state = task.grid
        reward = None
        cumulative_reward = 0

        while iteration < MIN_ITERATIONS or episode < MIN_EPISODES:
                        
            action = agent.interact(reward, board_state)

            if task.move(action, player)[1] == 50:
                print "Won!"
                break

            return_val, reward = task.move(action, player)

            if iteration < MIN_ITERATIONS:
                print np.rot90(task.grid)

                rewards_by_iteration.append(reward)
                if cumu_rewards_by_iteration == []:
                    cumu_rewards_by_iteration.append(reward)
                else:
                    cumu_rewards_by_iteration.append(cumu_rewards_by_iteration[-1] + reward)
                
            cumulative_reward += reward

            iteration += 1

        if episode < MIN_EPISODES:
            rewards_by_episode.append(cumulative_reward)
            if cumu_rewards_by_episode == []:
                cumu_rewards_by_episode.append(cumulative_reward)
            else:
                cumu_rewards_by_episode.append(cumu_rewards_by_episode[-1] + cumulative_reward)
        episode += 1
        
    return rewards_by_iteration, rewards_by_episode, cumu_rewards_by_iteration, cumu_rewards_by_episode

In [10]:
def play_game(board, p1, p2, q=False):
    """
    Runs Connect 4 game given simulator object and two agents (players)
    """
    reward = None
    
    if q == True:
        while True:
            print("p1")
            p1move = p1.interact(reward, board)
            print(p1move)
            if (p1move is None):
                board.print_grid()
                print("error")
                return -1
            p1result, reward = board.move(p1move, 1)
            print p1result
            if (p1result[0] == 1):
                print("player 1")
                board.print_grid()
                return 1
            elif (p1result[0] == -1):
                board.print_grid()
                print("error")
                return -1
            print("p2")
            p2move = p2.calc_next_move()
            print(p2move)
            if (p2move is None):
                board.print_grid()
                print("error")
                return -1
            p2result = board.move(p2move, -1)
            print p2result
            if (p2result[0] == 1):
                print("player 2")
                board.print_grid()
                return 1
            elif (p2result[0] == -1):
                board.print_grid()
                print("error")
                return -1
    
    else:
        while True:
            print("p1")
            p1move = p1.calc_next_move()
            print(p1move)
            if (p1move is None):
                board.print_grid()
                print("error")
                return -1
            p1result = board.move(p1move, 1)
            print p1result
            if (p1result[0] == 1):
                print("player 1")
                board.print_grid()
                return 1
            elif (p1result[0] == -1):
                board.print_grid()
                print("error")
                return -1
            print("p2")
            p2move = p2.calc_next_move()
            print(p2move)
            if (p2move is None):
                board.print_grid()
                print("error")
                return -1
            p2result = board.move(p2move, -1)
            print p2result
            if (p2result[0] == 1):
                print("player 2")
                board.print_grid()
                return 1
            elif (p2result[0] == -1):
                board.print_grid()
                print("error")
                return -1

In [11]:
class Random_Learner(object):
    """
    Implementation of Connect 4 agent that takes random moves at each action step
    """
    
    def __init__(self, board):
        self.board = board
        

    def calc_next_move(self):
        moves = self.board.next_possible_moves()
        return moves[random.randint(0, len(moves) - 1)]
        

In [12]:
depth = 2
N = 4
grid_size = 7
x = ConnectN(grid_size, N)
#p1 = Minimax_Learner(x, depth, N, 1) 
p2 = Random_Learner(x) 
p1 = Q_Learner(x)
play_game(x, p1, p2, True)

p1


ValueError: Input must >= 2-d.